## ЛР 4
Apache Spark

[источник набора данных](https://www.kaggle.com/datasets/mrdaniilak/russia-real-estate-2021/data)

### Описание данных

- date - дата публикации объявления;
- price - цена в рублях;
- level - этаж квартиры;
- levels - количество этажей в доме;
- rooms - количество жилых комнат (если значение равно "-1", то это означает "квартира-студия");
- area - общая площадь квартиры;
- kitchen_area - площадь кухни;
- geo_lat - широта;
- geo_lon - долгота;
- building_type - тип фасада (0 - не знаю; 1 - другое; 2 - панельный; 3 - монолитный; 4 - кирпичный; 5 - блочный; 6 - деревянный);
- object_type - тип квартиры (0 - вторичный рынок недвижимости; 2 - новостройка);
- postal_code - почтовый индекс;
- street_id - идентификатор улицы;
- id_region - номер региона в России (всего 85 субъектов);
- house_id - идентификатор дома.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("pyspark")\
                    .getOrCreate()

### Импорт данных

In [3]:
DATA_ESTATE_ADS_PATH = "data/input_data.csv"
DATA_REGIONS_PATH = "data/regions.txt"
OUT_DATA_ESTATE_ADS_PATH = "data/out_data.csv"

#### структурированные данные

In [4]:
df_estate_ads = (
    spark.read
    .option("delimiter", ";")
    .option("header", True)
    .option("inferSchema", True)
    .csv(DATA_ESTATE_ADS_PATH)
)

df_estate_ads.printSchema()

root
 |-- date: date (nullable = true)
 |-- price: long (nullable = true)
 |-- level: integer (nullable = true)
 |-- levels: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- area: double (nullable = true)
 |-- kitchen_area: double (nullable = true)
 |-- geo_lat: double (nullable = true)
 |-- geo_lon: double (nullable = true)
 |-- building_type: integer (nullable = true)
 |-- object_type: integer (nullable = true)
 |-- postal_code: integer (nullable = true)
 |-- street_id: integer (nullable = true)
 |-- id_region: integer (nullable = true)
 |-- house_id: integer (nullable = true)



In [5]:
df_estate_ads.show(5)

+----------+--------+-----+------+-----+----+------------+----------+----------+-------------+-----------+-----------+---------+---------+--------+
|      date|   price|level|levels|rooms|area|kitchen_area|   geo_lat|   geo_lon|building_type|object_type|postal_code|street_id|id_region|house_id|
+----------+--------+-----+------+-----+----+------------+----------+----------+-------------+-----------+-----------+---------+---------+--------+
|2021-01-01| 2451300|   15|    31|    1|30.3|         0.0|56.7801124|60.6993548|            0|          2|     620000|     NULL|       66| 1632918|
|2021-01-01| 1450000|    5|     5|    1|33.0|         6.0|44.6081542|40.1383814|            0|          0|     385000|     NULL|        1|    NULL|
|2021-01-01|10700000|    4|    13|    3|85.0|        12.0|55.5400601|37.7251124|            3|          0|     142701|   242543|       50|  681306|
|2021-01-01| 3100000|    3|     5|    3|82.0|         9.0|44.6081542|40.1383814|            0|          0|     3

In [6]:
df_estate_ads.describe().show()

+-------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|             price|            level|           levels|             rooms|             area|      kitchen_area|          geo_lat|           geo_lon|     building_type|       object_type|       postal_code|        street_id|         id_region|          house_id|
+-------+------------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|            490772|           490772|           490772|            490772|           490772|            490772|           490772|            490771|            490771|           

##### пропуски

In [7]:
df_estate_ads.select([
    F.count(F.when(F.col(c).isNull(), c))
    .alias(c) for c in df_estate_ads.columns
]).show(vertical=True)

-RECORD 0---------------
 date          | 0      
 price         | 0      
 level         | 0      
 levels        | 0      
 rooms         | 0      
 area          | 0      
 kitchen_area  | 0      
 geo_lat       | 0      
 geo_lon       | 1      
 building_type | 1      
 object_type   | 1      
 postal_code   | 22435  
 street_id     | 165736 
 id_region     | 1      
 house_id      | 123107 



In [8]:
df_estate_ads = df_estate_ads.dropna()

##### дубликаты

In [9]:
df_estate_ads.groupBy(df_estate_ads.columns)\
             .count()\
             .filter("count > 1")\
             .agg(F.sum("count") - F.count("*"))\
             .show()

+-----------------------+
|(sum(count) - count(1))|
+-----------------------+
|                   6415|
+-----------------------+



In [10]:
df_estate_ads = df_estate_ads.dropDuplicates()

#### неструктурированные данные

In [11]:
df_regions = (
    spark.read
    .text(DATA_REGIONS_PATH)
    .withColumn("id_region", F.regexp_extract("value", r"Код региона (\d+)", 1).cast("int"))\
    .withColumn("region_name", F.regexp_extract("value", r"соответствует (.+)", 1))
)

df_regions.printSchema()

root
 |-- value: string (nullable = true)
 |-- id_region: integer (nullable = true)
 |-- region_name: string (nullable = true)



In [12]:
df_regions = df_regions.select("id_region", "region_name")
df_regions.show(5, truncate=False)

+---------+---------------------+
|id_region|region_name          |
+---------+---------------------+
|78       |Санкт-Петербург      |
|47       |Ленинградская область|
|77       |Москва               |
|92       |Севастополь          |
|50       |Московская область   |
+---------+---------------------+
only showing top 5 rows



##### объединение

In [13]:
df_estate_ads = df_estate_ads.join(
    df_regions,
    how="inner",
    on="id_region"
)

df_estate_ads.printSchema()

root
 |-- id_region: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- price: long (nullable = true)
 |-- level: integer (nullable = true)
 |-- levels: integer (nullable = true)
 |-- rooms: integer (nullable = true)
 |-- area: double (nullable = true)
 |-- kitchen_area: double (nullable = true)
 |-- geo_lat: double (nullable = true)
 |-- geo_lon: double (nullable = true)
 |-- building_type: integer (nullable = true)
 |-- object_type: integer (nullable = true)
 |-- postal_code: integer (nullable = true)
 |-- street_id: integer (nullable = true)
 |-- house_id: integer (nullable = true)
 |-- region_name: string (nullable = true)



In [14]:
df_estate_ads.show(1, vertical=True)

-RECORD 0---------------------------
 id_region     | 23                 
 date          | 2021-01-01         
 price         | 5150000            
 level         | 9                  
 levels        | 10                 
 rooms         | 4                  
 area          | 83.6               
 kitchen_area  | 10.3               
 geo_lat       | 44.872946          
 geo_lon       | 37.3296834         
 building_type | 2                  
 object_type   | 0                  
 postal_code   | 353454             
 street_id     | 543620             
 house_id      | 2307522            
 region_name   | Краснодарский край 
only showing top 1 row



### аналитика

In [15]:
# средняя цена по регионам
df_estate_ads.groupBy("region_name")\
             .agg(F.format_number(F.avg("price"), 2).alias("avg_price"))\
             .orderBy("avg_price", ascending=False)\
             .show(10, truncate=False)

+-----------------------+------------+
|region_name            |avg_price   |
+-----------------------+------------+
|Республика Бурятия     |8,665,883.15|
|Московская область     |6,561,304.84|
|Севастополь            |6,499,573.34|
|Калужская область      |5,979,123.66|
|Ленинградская область  |5,611,646.67|
|Краснодарский край     |5,520,742.49|
|Калининградская область|5,209,131.57|
|Амурская область       |5,147,476.97|
|Приморский край        |4,834,785.60|
|Республика Тыва        |4,778,210.53|
+-----------------------+------------+
only showing top 10 rows



In [16]:
# самые дорогие объявления
df_estate_ads.orderBy(F.desc("price"))\
             .select("date", "price", "area", "region_name")\
             .show(5, truncate=False)

+----------+----------+----+------------------+
|date      |price     |area|region_name       |
+----------+----------+----+------------------+
|2021-01-15|6500000000|95.8|Республика Бурятия|
|2021-01-27|6230000128|57.0|Тюменская область |
|2021-01-07|5500000256|61.5|Калужская область |
|2021-01-28|5310000128|60.0|Тюменская область |
|2021-01-23|3850000000|71.0|Калужская область |
+----------+----------+----+------------------+
only showing top 5 rows



In [17]:
# средняя цена от типа здания
df_estate_ads.groupBy("building_type")\
             .agg(F.format_number(F.avg("price"), 2).alias("avg_price"))\
             .orderBy("building_type")\
             .show()

+-------------+-------------+
|building_type|    avg_price|
+-------------+-------------+
|            0| 7,831,731.55|
|            1| 5,835,191.56|
|            2| 3,835,645.47|
|            3|11,419,633.73|
|            4| 4,640,876.04|
|            5| 4,503,950.24|
|            6| 2,430,807.10|
+-------------+-------------+



### экспорт данных

In [18]:
df_estate_ads.write\
             .mode("overwrite")\
             .format("csv")\
             .option("header", True)\
             .save(OUT_DATA_ESTATE_ADS_PATH)